# Import Customized modules and Create ImageJ Object

In [ ]:
import os
import sys
import re
from pathlib import Path

abs_module_path = Path("./../../").resolve()
if (abs_module_path.exists()) and (str(abs_module_path) not in sys.path):
    sys.path.append(str(abs_module_path)) # add path to scan customized module

from modules.data.lif import LIFNameChecker, scan_lifs_under_dir
from modules.data.get_path import get_fiji_local_path, get_lif_scan_root
from modules.misc.logger import init_logger
from modules.misc.utils import get_repo_root, load_config

Initialize `Customized Object`

In [ ]:
logger = init_logger(r"Check Lif Name")
lif_name_checker = LIFNameChecker()

In [ ]:
repo_root = get_repo_root()
print(f"Repository: '{repo_root}'")

Load `db_path_plan.toml`

In [ ]:
dbpp_config = load_config("db_path_plan.toml")

Load `0.1.check_lif_name.toml`

In [ ]:
config = load_config("0.1.check_lif_name.toml")
nasdl_type    = config["data_nasdl"]["type"]
nasdl_batches = config["data_nasdl"]["batches"]

Initialize `PyImageJ`

In [ ]:
import jpype               # Import module
import jpype.imports       # Enable Java imports
from jpype.types import *  # Pull in types
import scyjava as sj       # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej              # pyimagej

""" Configurations """
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). 
#       If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

""" Get path of Fiji(ImageJ) """
fiji_local = get_fiji_local_path(dbpp_config)

""" Different methods to start ImageJ """
# ij = imagej.init(fiji_local) # Same as "ij = imagej.init(fiji_local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(fiji_local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(fiji_local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

""" Print Fiji version """
print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.54b

Load other `Java components`

In [ ]:
# Bio-Format Reader
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# Run

In [ ]:
lif_scan_root = get_lif_scan_root(dbpp_config, config, logger=logger)
lif_path_list = scan_lifs_under_dir(lif_scan_root, nasdl_batches, logger=logger)


for i, lif_path in enumerate(lif_path_list):
    
    logger.info(f"Processing ... {i+1}/{len(lif_path_list)}")
    logger.info(f'LIF_FILE : {lif_path}')
    
    
    """ normalize LIF name """
    file_name = lif_path.split(os.sep)[-1].split(".")[0]
    file_name_list = re.split(" |_|-", file_name)
    assert len(file_name_list) == 4, f"file_name format error, current : '{file_name}', expect like : '20221125_AI005_palmskin_10dpf.lif'"
    file_name = "_".join(file_name_list)
    
    """ Get number of images in LIF file """
    Reader.setId(lif_path)
    seriesCount = Reader.getSeriesCount()
    
    
    for idx in range(seriesCount):
        
        SeriesNum = idx+1
        
        """ read image """
        ij.IJ.run("Bio-Formats Importer", f"open='{lif_path}' color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{SeriesNum}")
        img = ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>
        img.hide()
        
        """ Get image name """
        image_name = str(img.getProp("Image name"))
        
        
        """ BrightField_RAW """
        if nasdl_type == "BrightField_RAW":
            if "Before_20221109" in lif_path:
                lif_name_checker.check_image_name(image_name, "old_bf")
            else: 
                lif_name_checker.check_image_name(image_name, "new_bf")
        
        """ PalmSkin_RAW """
        if nasdl_type == "PalmSkin_RAW":
            if "Before_20221109" in lif_path:
                lif_name_checker.check_image_name(image_name, "old_rgb")
            else: 
                lif_name_checker.check_image_name(image_name, "new_rgb")
        
        
        """ Concat LIF and image name """
        seN = f"{file_name} - {image_name}"
        
        """ Get image dimension """
        img_dimensions = img.getDimensions()
        logger.info(f"series {SeriesNum:{len(str(seriesCount))}}/{seriesCount} : '{seN}' , "
                    f"Dimensions : {img_dimensions} ( width, height, channels, slices, frames )")
        
        """ Print ERRORs after checking 'image name' """
        try:
            logger.info(f"       ##### {lif_name_checker.check_dict['failed message']}")
        except KeyError:
            pass
    
    
    logger.info("\n")


logger.info(" -- finished -- ")